In [14]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
%matplotlib inline

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [15]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28 #image size
n_noise = 128 # noise size

In [16]:
X = tf.placeholder(tf.float32,[None, n_input]) # 원래 data
Z = tf.placeholder(tf.float32,[None,n_noise]) # noise

In [17]:
#generat 변수들
#은닉층 출력을 위한 가중치,편향
G_W1 = tf.Variable(tf.random_normal([n_noise,n_hidden],stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
#출력층 출력을 위한 가중치,편향
G_W2 = tf.Variable(tf.random_normal([n_hidden,n_input],stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [18]:
#difcriminator 변수들
#은닉층 출력을 위한 가중치,편향
D_W1 = tf.Variable(tf.random_normal([n_input,n_hidden],stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
#출력층 출력을 위한 가중치,편향
D_W2 = tf.Variable(tf.random_normal([n_hidden,1],stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [23]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z,G_W1)+G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,G_W2)+G_b2)
    return output

In [25]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs,D_W1)+D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2) # output을 0과 1 사이의 값으로 만들어줌
    return output

In [21]:
#무작위 noise를 batch_size만큼 만들어줌
def get_noise(batch_size,n_noise):
    return np.random.normal(size=(batch_size,n_noise))

In [26]:
G = generator(Z)# noise로 부터 생성된 이미지
D_gene = discriminator(G) #가짜 이미지 판별값
D_real = discriminator(X) #진짜 이미지 판별값

In [27]:
# discriminator loss
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))

In [28]:
# generator loss
loss_G = tf.reduce_mean(tf.log(D_gene))

In [30]:
#각각 신경망 변수들 구분해줌
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [31]:
# training 방법 설정
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,var_list=D_var_list)#maximise 하기위해 loss에 음수 붙여줌
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list=G_var_list)

In [32]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D,loss_val_G=0,0 #loss D와 loss_G의 결과값을 받을 변수

In [ ]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size,n_noise)
        
        _,loss_val_D = sess.run([train_D,loss_D],feed_dict={X:batch_xs, Z:noise})
        _,loss_val_G = sess.run([train_G,loss_G],feed_dict={Z:noise})
        
        print('Epoch:', ' %04d' %epoch,'D loss:{:.4}'.format(loss_val_D),'G loss:{:.4}'.format(loss_val_G))
        # 생성된 이미지 저장
        if epoch == 0 or (epoch +1) % 10 == 0:
            sample_size = 10
            noise = get_noise(sample_size, n_noise) # size 128 noise 를 10개씩 만듬
            samples = sess.run(G, feed_dict={Z: noise})
            
            fig, ax = plt.subplots(1, sample_size, figsize = {sample_size, 1})
            
            for i in range(sample_size):
                ax[i].set_axis_off()
                ax[i].imshow(np.reshape(samples[i],(28,28)))
                
            plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
            plt.close(fig)
        print('최적화 완료!')

Epoch:  0000 D loss:-1.383 G loss:-0.7332
Epoch:  0000 D loss:-1.351 G loss:-0.7623
Epoch:  0000 D loss:-1.327 G loss:-0.7894
Epoch:  0000 D loss:-1.306 G loss:-0.8159
Epoch:  0000 D loss:-1.285 G loss:-0.843
Epoch:  0000 D loss:-1.267 G loss:-0.8709
Epoch:  0000 D loss:-1.245 G loss:-0.9
Epoch:  0000 D loss:-1.225 G loss:-0.9301
Epoch:  0000 D loss:-1.207 G loss:-0.9612
Epoch:  0000 D loss:-1.185 G loss:-0.9939
Epoch:  0000 D loss:-1.162 G loss:-1.027
Epoch:  0000 D loss:-1.14 G loss:-1.061
Epoch:  0000 D loss:-1.117 G loss:-1.097
Epoch:  0000 D loss:-1.096 G loss:-1.134
Epoch:  0000 D loss:-1.075 G loss:-1.171
Epoch:  0000 D loss:-1.049 G loss:-1.21
Epoch:  0000 D loss:-1.027 G loss:-1.249
Epoch:  0000 D loss:-1.005 G loss:-1.287
Epoch:  0000 D loss:-0.9751 G loss:-1.326
Epoch:  0000 D loss:-0.9546 G loss:-1.365
Epoch:  0000 D loss:-0.9295 G loss:-1.406
Epoch:  0000 D loss:-0.9025 G loss:-1.446
Epoch:  0000 D loss:-0.8735 G loss:-1.489
Epoch:  0000 D loss:-0.8452 G loss:-1.532
Epoch:

Epoch:  0000 D loss:-0.7006 G loss:-2.332
Epoch:  0000 D loss:-0.6765 G loss:-2.287
Epoch:  0000 D loss:-0.6758 G loss:-2.349
Epoch:  0000 D loss:-0.6999 G loss:-2.207
Epoch:  0000 D loss:-0.6722 G loss:-2.263
Epoch:  0000 D loss:-0.7229 G loss:-2.282
Epoch:  0000 D loss:-0.6921 G loss:-2.432
Epoch:  0000 D loss:-0.7788 G loss:-2.287
Epoch:  0000 D loss:-0.7197 G loss:-2.148
Epoch:  0000 D loss:-0.6887 G loss:-2.48
Epoch:  0000 D loss:-0.7389 G loss:-1.997
Epoch:  0000 D loss:-0.7233 G loss:-2.088
Epoch:  0000 D loss:-0.6783 G loss:-2.181
Epoch:  0000 D loss:-0.7812 G loss:-1.939
Epoch:  0000 D loss:-0.8196 G loss:-1.982
Epoch:  0000 D loss:-0.6878 G loss:-2.137
Epoch:  0000 D loss:-0.7679 G loss:-2.025
Epoch:  0000 D loss:-0.7143 G loss:-2.086
Epoch:  0000 D loss:-0.7681 G loss:-1.859
Epoch:  0000 D loss:-0.7853 G loss:-1.994
Epoch:  0000 D loss:-0.7459 G loss:-2.111
Epoch:  0000 D loss:-0.7427 G loss:-1.945
Epoch:  0000 D loss:-0.7405 G loss:-1.965
Epoch:  0000 D loss:-0.749 G loss:-

Epoch:  0000 D loss:-0.7097 G loss:-1.74
Epoch:  0000 D loss:-0.7043 G loss:-1.758
Epoch:  0000 D loss:-0.6971 G loss:-1.786
Epoch:  0000 D loss:-0.7517 G loss:-1.795
Epoch:  0000 D loss:-0.7688 G loss:-1.795
Epoch:  0000 D loss:-0.6887 G loss:-1.754
Epoch:  0000 D loss:-0.726 G loss:-1.723
Epoch:  0000 D loss:-0.667 G loss:-1.747
Epoch:  0000 D loss:-0.682 G loss:-1.754
Epoch:  0000 D loss:-0.7317 G loss:-1.779
Epoch:  0000 D loss:-0.6954 G loss:-1.787
Epoch:  0000 D loss:-0.7776 G loss:-1.788
Epoch:  0000 D loss:-0.6738 G loss:-1.782
Epoch:  0000 D loss:-0.699 G loss:-1.818
Epoch:  0000 D loss:-0.6909 G loss:-1.8
Epoch:  0000 D loss:-0.7364 G loss:-1.789
Epoch:  0000 D loss:-0.6977 G loss:-1.755
Epoch:  0000 D loss:-0.7279 G loss:-1.754
Epoch:  0000 D loss:-0.72 G loss:-1.727
Epoch:  0000 D loss:-0.7116 G loss:-1.766
Epoch:  0000 D loss:-0.6679 G loss:-1.759
Epoch:  0000 D loss:-0.7133 G loss:-1.759
Epoch:  0000 D loss:-0.7067 G loss:-1.791
Epoch:  0000 D loss:-0.7168 G loss:-1.789
E

Epoch:  0001 D loss:-0.5182 G loss:-2.085
Epoch:  0001 D loss:-0.4874 G loss:-2.072
Epoch:  0001 D loss:-0.4367 G loss:-2.08
Epoch:  0001 D loss:-0.4107 G loss:-2.12
Epoch:  0001 D loss:-0.472 G loss:-2.16
Epoch:  0001 D loss:-0.4524 G loss:-2.187
Epoch:  0001 D loss:-0.4336 G loss:-2.243
Epoch:  0001 D loss:-0.5369 G loss:-2.245
Epoch:  0001 D loss:-0.4037 G loss:-2.25
Epoch:  0001 D loss:-0.4837 G loss:-2.265
Epoch:  0001 D loss:-0.3923 G loss:-2.259
Epoch:  0001 D loss:-0.4221 G loss:-2.278
Epoch:  0001 D loss:-0.4662 G loss:-2.25
Epoch:  0001 D loss:-0.3482 G loss:-2.269
Epoch:  0001 D loss:-0.4645 G loss:-2.283
Epoch:  0001 D loss:-0.4399 G loss:-2.256
Epoch:  0001 D loss:-0.368 G loss:-2.283
Epoch:  0001 D loss:-0.3938 G loss:-2.296
Epoch:  0001 D loss:-0.4833 G loss:-2.282
Epoch:  0001 D loss:-0.4119 G loss:-2.262
Epoch:  0001 D loss:-0.4024 G loss:-2.278
Epoch:  0001 D loss:-0.3458 G loss:-2.279
Epoch:  0001 D loss:-0.3715 G loss:-2.294
Epoch:  0001 D loss:-0.4228 G loss:-2.335

Epoch:  0001 D loss:-0.2678 G loss:-2.457
Epoch:  0001 D loss:-0.3046 G loss:-2.465
Epoch:  0001 D loss:-0.2545 G loss:-2.469
Epoch:  0001 D loss:-0.2556 G loss:-2.512
Epoch:  0001 D loss:-0.2843 G loss:-2.509
Epoch:  0001 D loss:-0.2806 G loss:-2.537
Epoch:  0001 D loss:-0.2767 G loss:-2.543
Epoch:  0001 D loss:-0.2815 G loss:-2.535
Epoch:  0001 D loss:-0.2491 G loss:-2.546
Epoch:  0001 D loss:-0.2878 G loss:-2.549
Epoch:  0001 D loss:-0.2607 G loss:-2.544
Epoch:  0001 D loss:-0.2655 G loss:-2.549
Epoch:  0001 D loss:-0.2567 G loss:-2.548
Epoch:  0001 D loss:-0.2875 G loss:-2.562
Epoch:  0001 D loss:-0.2706 G loss:-2.554
Epoch:  0001 D loss:-0.2632 G loss:-2.55
Epoch:  0001 D loss:-0.2622 G loss:-2.545
Epoch:  0001 D loss:-0.2721 G loss:-2.535
Epoch:  0001 D loss:-0.2719 G loss:-2.533
Epoch:  0001 D loss:-0.2728 G loss:-2.532
Epoch:  0001 D loss:-0.2608 G loss:-2.522
Epoch:  0001 D loss:-0.2587 G loss:-2.536
Epoch:  0001 D loss:-0.2378 G loss:-2.517
Epoch:  0001 D loss:-0.2499 G loss:

...

Epoch:  0079 D loss:-0.6642 G loss:-2.122
Epoch:  0079 D loss:-0.6846 G loss:-2.247
Epoch:  0079 D loss:-0.7272 G loss:-2.04
Epoch:  0079 D loss:-0.5814 G loss:-2.023
Epoch:  0079 D loss:-0.6893 G loss:-2.076
Epoch:  0079 D loss:-0.5663 G loss:-2.144
Epoch:  0079 D loss:-0.6396 G loss:-1.946
Epoch:  0079 D loss:-0.7133 G loss:-1.881
Epoch:  0079 D loss:-0.689 G loss:-1.923
Epoch:  0079 D loss:-0.7914 G loss:-1.909
Epoch:  0079 D loss:-0.6757 G loss:-1.929
Epoch:  0079 D loss:-0.7349 G loss:-1.857
Epoch:  0079 D loss:-0.8075 G loss:-1.928
Epoch:  0079 D loss:-0.5884 G loss:-2.193
Epoch:  0079 D loss:-0.5522 G loss:-2.351
Epoch:  0079 D loss:-0.7418 G loss:-2.09
Epoch:  0079 D loss:-0.5803 G loss:-2.271
Epoch:  0079 D loss:-0.6536 G loss:-2.01
Epoch:  0079 D loss:-0.792 G loss:-1.948
Epoch:  0079 D loss:-0.6751 G loss:-2.111
Epoch:  0079 D loss:-0.721 G loss:-1.955
Epoch:  0080 D loss:-0.6909 G loss:-2.021
Epoch:  0080 D loss:-0.7225 G loss:-1.857
Epoch:  0080 D loss:-0.6965 G loss:-1.98

Epoch:  0080 D loss:-0.7489 G loss:-2.142
Epoch:  0080 D loss:-0.6331 G loss:-2.086
Epoch:  0080 D loss:-0.7913 G loss:-1.992
Epoch:  0080 D loss:-0.625 G loss:-2.033
Epoch:  0080 D loss:-0.6282 G loss:-2.122
Epoch:  0080 D loss:-0.5916 G loss:-2.246
Epoch:  0080 D loss:-0.7582 G loss:-2.035
Epoch:  0080 D loss:-0.6488 G loss:-1.947
Epoch:  0080 D loss:-0.9434 G loss:-1.71
Epoch:  0080 D loss:-0.5676 G loss:-1.823
Epoch:  0080 D loss:-0.6329 G loss:-1.949
Epoch:  0080 D loss:-0.6327 G loss:-2.067
Epoch:  0080 D loss:-0.7152 G loss:-1.968
Epoch:  0080 D loss:-0.7321 G loss:-1.909
Epoch:  0080 D loss:-0.6558 G loss:-1.933
Epoch:  0080 D loss:-0.6082 G loss:-2.022
Epoch:  0080 D loss:-0.6378 G loss:-2.073
Epoch:  0080 D loss:-0.5678 G loss:-2.376
Epoch:  0080 D loss:-0.5735 G loss:-2.193
Epoch:  0080 D loss:-0.5619 G loss:-2.271
Epoch:  0080 D loss:-0.5781 G loss:-2.074
Epoch:  0080 D loss:-0.8184 G loss:-2.173
Epoch:  0080 D loss:-0.8252 G loss:-1.987
Epoch:  0080 D loss:-0.7228 G loss:-

Epoch:  0080 D loss:-0.7213 G loss:-1.905
Epoch:  0080 D loss:-0.7277 G loss:-1.924
Epoch:  0080 D loss:-0.603 G loss:-2.151
Epoch:  0080 D loss:-0.6497 G loss:-2.141
Epoch:  0080 D loss:-0.6081 G loss:-2.174
Epoch:  0080 D loss:-0.6222 G loss:-2.016
Epoch:  0080 D loss:-0.6091 G loss:-2.143
Epoch:  0080 D loss:-0.6699 G loss:-2.107
Epoch:  0080 D loss:-0.6184 G loss:-2.285
Epoch:  0080 D loss:-0.7148 G loss:-2.059
Epoch:  0080 D loss:-0.7093 G loss:-2.079
Epoch:  0080 D loss:-0.5817 G loss:-2.213
Epoch:  0080 D loss:-0.6687 G loss:-1.981
Epoch:  0080 D loss:-0.7194 G loss:-1.987
Epoch:  0080 D loss:-0.6916 G loss:-1.875
Epoch:  0080 D loss:-0.5664 G loss:-1.957
Epoch:  0080 D loss:-0.7423 G loss:-1.925
Epoch:  0080 D loss:-0.7422 G loss:-2.072
Epoch:  0080 D loss:-0.6738 G loss:-1.957
Epoch:  0080 D loss:-0.7398 G loss:-2.038
Epoch:  0080 D loss:-0.7929 G loss:-1.821
Epoch:  0080 D loss:-0.7449 G loss:-1.935
Epoch:  0080 D loss:-0.7042 G loss:-2.196
Epoch:  0080 D loss:-0.6703 G loss:

Epoch:  0081 D loss:-0.6406 G loss:-1.982
Epoch:  0081 D loss:-0.6373 G loss:-2.179
Epoch:  0081 D loss:-0.758 G loss:-2.024
Epoch:  0081 D loss:-0.5803 G loss:-2.21
Epoch:  0081 D loss:-0.5548 G loss:-2.09
Epoch:  0081 D loss:-0.7247 G loss:-2.136
Epoch:  0081 D loss:-0.6406 G loss:-2.118
Epoch:  0081 D loss:-0.8437 G loss:-1.978
Epoch:  0081 D loss:-0.816 G loss:-2.077
Epoch:  0081 D loss:-0.7091 G loss:-1.864
Epoch:  0081 D loss:-0.7551 G loss:-1.6
Epoch:  0081 D loss:-0.8047 G loss:-1.64
Epoch:  0081 D loss:-0.578 G loss:-1.972
Epoch:  0081 D loss:-0.7055 G loss:-1.848
Epoch:  0081 D loss:-0.8296 G loss:-1.954
Epoch:  0081 D loss:-0.8403 G loss:-1.903
Epoch:  0081 D loss:-0.7559 G loss:-1.86
Epoch:  0081 D loss:-0.6911 G loss:-2.016
Epoch:  0081 D loss:-0.7047 G loss:-2.16
Epoch:  0081 D loss:-0.7574 G loss:-2.145
Epoch:  0081 D loss:-0.7289 G loss:-2.118
Epoch:  0081 D loss:-0.7382 G loss:-2.079
Epoch:  0081 D loss:-0.851 G loss:-1.862
Epoch:  0081 D loss:-0.6724 G loss:-1.788
Epo

Epoch:  0081 D loss:-0.8155 G loss:-1.797
Epoch:  0081 D loss:-0.7759 G loss:-1.74
Epoch:  0081 D loss:-0.8095 G loss:-1.939
Epoch:  0081 D loss:-0.6846 G loss:-2.001
Epoch:  0081 D loss:-0.7933 G loss:-1.998
Epoch:  0081 D loss:-0.7225 G loss:-2.293
Epoch:  0081 D loss:-0.8831 G loss:-2.219
Epoch:  0081 D loss:-0.8753 G loss:-2.105
Epoch:  0081 D loss:-0.7503 G loss:-2.335
Epoch:  0081 D loss:-0.7795 G loss:-1.968
Epoch:  0081 D loss:-0.882 G loss:-2.039
Epoch:  0081 D loss:-0.5906 G loss:-1.92
Epoch:  0081 D loss:-0.7016 G loss:-1.878
Epoch:  0081 D loss:-0.7224 G loss:-1.791
Epoch:  0081 D loss:-0.7412 G loss:-1.841
Epoch:  0081 D loss:-0.8155 G loss:-1.555
Epoch:  0081 D loss:-0.6993 G loss:-1.699
Epoch:  0081 D loss:-0.8023 G loss:-1.909
Epoch:  0081 D loss:-0.7321 G loss:-2.003
Epoch:  0081 D loss:-0.6747 G loss:-1.955
Epoch:  0081 D loss:-0.5848 G loss:-1.936
Epoch:  0081 D loss:-0.7291 G loss:-1.741
Epoch:  0081 D loss:-0.7589 G loss:-1.942
Epoch:  0081 D loss:-0.7693 G loss:-2